In [1]:
workspace_PATH = 'workspace'
scripts_PATH = 'scripts'
apimodel_PATH = 'models'
annotation_PATH = workspace_PATH + '/annotations'
image_PATH = workspace_PATH +'/images'
model_PATH = workspace_PATH +'/models'
pretrained_model_PATH = workspace_PATH +'/pre-trained-models'
config_PATH = model_PATH + '/my_ssd_modnet/pipeline.config'
checkpoint_PATH = model_PATH + '/my_ssd_modenet/'

In [2]:
labels = [{'name':'face_with_mask','id':1}, {'name': 'face_no_mask','id':2}]

In [3]:
labels

[{'name': 'face_with_mask', 'id': 1}, {'name': 'face_no_mask', 'id': 2}]

In [4]:
with open(annotation_PATH + '/label_map.pbtxt','w' ) as f:
    for label in labels:
        f.write('item[\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [5]:
!python {scripts_PATH+ '/generate_tfrecord.py'} -x {image_PATH + '/train'} -l {annotation_PATH + '/label_map.pbtxt'} -o {annotation_PATH + '/train.record'}

Traceback (most recent call last):
  File "/home/dheeraj/Desktop/RWTH/Coding/CV/scripts/generate_tfrecord.py", line 61, in <module>
    label_map = label_map_util.load_labelmap(args.labels_path)
  File "/home/dheeraj/anaconda3/envs/Coding/lib/python3.9/site-packages/object_detection/utils/label_map_util.py", line 132, in load_labelmap
    with tf.gfile.GFile(path, 'r') as fid:
AttributeError: module 'tensorflow' has no attribute 'gfile'
